In [1]:
import funcoes as fc
import numpy as np
import os
import pandas as pd
import pyautogui
import pyperclip
import time
import tkinter as tk
import undetected_chromedriver as uc
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tabulate import tabulate
from tkinter import scrolledtext
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#INSTALA O CHROME DRIVEr MAIS ATUALIZADO
# Desativa o bloqueio de pop-ups
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking") 
#DEFINE O TEMPO DE EXECUÇÃO PARA CADA COMANDO DO PYAUTOGUI
pyautogui.PAUSE = 0.7
#INICIA O NAVEGADOR
# Caminho do ChromeDriver local
# chrome_driver_path = r"main\chromedriver-win64\chromedriver.exe"  # Altere para o caminho correto do seu ChromeDriver
# Configura o serviço do ChromeDriver
servico = Service(ChromeDriverManager().install())
# Inicia o navegador Chrome
navegador = webdriver.Chrome(service=servico, options=options)
navegador.maximize_window()
#ENTRA NO SEI
navegador.get('https://sei.anatel.gov.br/')


In [29]:
navegador.switch_to.window(janela_principal)

In [ ]:
processosAssinados = '53500.062842/2024-66'
janela_principal = navegador.current_window_handle
textoCancelamento = f"""Prezado(a) Senhor(a),

    Informamos que a solicitação de cancelamento de homologação foi deferida.


    FAVOR NÃO RESPONDER ESTE E-MAIL. 

    

    Atenciosamente,

    ORCN - Gerência de Certificação e Numeração

    SOR - Superintendência de Outorga e Recursos à Prestação

    Anatel - Agência Nacional de Telecomunicações"""

print('\n')
print(f'Concluindo processo nº {processosAssinados}')
fc.vai_para_processo(navegador, processosAssinados)
time.sleep(1)
#PEGA DADOS DA DECLARACAO DE CONFORMIDADE
navegador.switch_to.frame('ifrArvore')

#CONFERE SE EXISTE A DECLARACAO DE CONFORMIDADE OU UMA PASTA
if fc.check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade", navegador):
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)
elif fc.check_element_exists(By.XPATH, '//*[@id="spanPASTA1"]', navegador):
    navegador.find_element(By.XPATH, '//*[@id="spanPASTA1"]').click()
    time.sleep(1)
    navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
    time.sleep(0.3)

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
navegador.switch_to.frame('ifrVisualizacao')

#ARMAZENA EMAIL DO SOLICITANTE
emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        print('Encontrado despacho de cancelamento')
        #VERIFICA SE EXISTE ASSINATURA DO GERENTE
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        navegador.switch_to.frame('ifrVisualizacao')
        if fc.check_element_exists(By.XPATH, "/html/body/div[1]/table[1]", navegador):
            navegador.switch_to.default_content()
            navegador.switch_to.frame('ifrConteudoVisualizacao')
            #CLICA NO ICONE DE ENVIO DE EMAIL
            fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/email_enviar.svg?18')]")
            #clica_noelemento(navegador, By.XPATH, '//*[@id="divArvoreAcoes"]/a[10]')
            time.sleep(1)
            #VAI PARA A JANELA MAIS RECENTE ABERTA
            navegador.switch_to.window(navegador.window_handles[-1])
            #time.sleep(1)
            #ABRE O DROPDOWN COM AS OPCOES DE EMAIL DA ANATEL
            select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selDe"]')))
            select = Select(select_element)
            #SELECIONA O EMAIL DA ANATEL
            select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
            #ESCREVE O EMAIL DO SOLICITANTE
            navegador.find_element(By.XPATH, '//*[@id="s2id_autogen1"]').send_keys(emailSol)
            time.sleep(1.2)
            #CLICA NO EMAIL DO SOLICITANTE
            fc.clica_noelemento(navegador, By.CLASS_NAME, 'select2-result-label')
            #INSERE ASSUNTO DO PROCESO
            navegador.find_element(By.ID, 'txtAssunto').send_keys(f"Processo SEI nº {processosAssinados}")
            #INSERE EMAIL COM CORPO DO EMAIL
            navegador.find_element(By.ID, 'txaMensagem').send_keys(textoCancelamento)
            #ENVIA EMAIL
            fc.clica_noelemento(navegador, By.XPATH,'//*[@id="divInfraBarraComandosInferior"]/button[1]')
            # navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
            #FECHA ALERTA DO NAVEGADOR
            alert = Alert(navegador)
            alert.accept()

            #RETORNA PARA A JANELA PRINCIPAL
            navegador.switch_to.window(janela_principal)
            fc.vai_para_processo(navegador, processosAssinados)
            time.sleep(1)
            #CLICA NO ICONE DE ANOTACAO
            navegador.switch_to.frame('ifrConteudoVisualizacao')
            #time.sleep(1)
            fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
            #navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[17]').click()
            #LIMPA O TEXTO DA ANOTACAO
            time.sleep(0.3)
            navegador.switch_to.frame('ifrVisualizacao')
            navegador.find_element(By.XPATH, '//*[@id="txaDescricao"]').clear()
            #SALVA ANOTACAO
            fc.clica_noelemento(navegador, By.XPATH,'//*[@id="divInfraBarraComandosSuperior"]/button')
            #CONCLUI PROCESSO
            navegador.switch_to.default_content
            navegador.switch_to.frame('ifrConteudoVisualizacao')
            fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/processo_concluir.svg?18')]")
            #clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[19]')
            navegador.switch_to.frame('ifrVisualizacao')
            fc.clica_noelemento(navegador, By.XPATH, '//*[@id="sbmSalvar"]')
            # navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[20]').click()
            print(f"Processo nº {processosAssinados}")
            time.sleep(0.3)
            navegador.switch_to.default_content()
        else:
            print(f"O despacho de cancelamento do processo {processosAssinados} ainda não foi assinado, execute o script novamente após conter a assinatura do gerente.")



Concluindo processo nº 53500.062842/2024-66
Encontrado despacho de cancelamento


NoSuchFrameException: Message: ifrConteudoVisualizacao


In [3]:
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')

if fc.check_element_exists(By.PARTIAL_LINK_TEXT, 'Despacho Decisório', navegador) and not fc.check_element_exists(By.XPATH, "//img[contains(@src, 'svg/marcador_verde.svg?18')]", navegador):
    print("O processo já foi despachado!")
    
elif fc.check_element_exists(By.PARTIAL_LINK_TEXT, 'Despacho Decisório', navegador) and fc.check_element_exists(By.XPATH, "//img[contains(@src, 'svg/marcador_verde.svg?18')]", navegador):
    tipo_processo = 'Reaberto'
    print('processo reaberto')

processo reaberto


In [39]:
fc.vai_para_processo(navegador, processosAssinados)
navegador.switch_to.frame('ifrConteudoVisualizacao')
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/processo_concluir.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[19]')
navegador.switch_to.frame('ifrVisualizacao')
fc.clica_noelemento(navegador, By.XPATH, '//*[@id="sbmSalvar"]')
# navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[20]').click()
print(f"Processo nº {processosAssinados}")

Processo nº 53500.062842/2024-66


In [9]:
fc.analisa(navegador, '53500.104031/2024-40', 'André Jacinto Rodrigues', fc.corrige_planilha(planilhaDrones, True), fc.corrige_planilha(planilhaRadios, False))

Não foi possível encontrar a Declaração de Conformidade.
Célula encontrada: 53500.104031/2024-40


In [11]:
processo = '53500.070427/2024-86'

In [14]:
fc.retorna_pagina_inicial(navegador, processo)
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        print('Despacho Decisório de Cancelamento encontrado.\n')

        time.sleep(1)
        #CLICA NO ICONE DE LEGO
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
        
        #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
        time.sleep(1.5)
        navegador.switch_to.frame('ifrVisualizacao')
        select_element = navegador.find_element(By.ID, 'selBloco')
        select = Select(select_element)
        #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
        for opcao in select.options:
            if "Despachos para Drones aprovados" in opcao.text:
                select.select_by_visible_text(opcao.text)
                break
        time.sleep(0.5)
        #CLICA NO BOTAO DE INCLUIR NO BLOCO
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
        #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()






Despacho Decisório de Cancelamento encontrado.



In [16]:
#VOLTA PARA PAGINA INICIAL DO PROCESSO
fc.retorna_pagina_inicial(navegador, processo)
time.sleep(1)
#ADICIONA NOTA PARA AGUARDAR ASSINATURA
#CLICA NO ICONE DE ANOTACAO
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
time.sleep(0.1)
#INSERE O TEXTO DA ANOTACAO
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.ID, 'txaDescricao').send_keys('Pedido de Cancelamento')
#SALVA O TEXTO
navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
#DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES
fc.preenche_planilhageral(processo, 'André Jacinto Rodrigues', situacao="Cancelado")

Célula encontrada: Exigência


In [3]:
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/marcador_gerenciar.svg?18')]")
#navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[24]').click()
navegador.switch_to.frame('ifrVisualizacao')


In [5]:
navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
fc.clica_noelemento(navegador, By.PARTIAL_LINK_TEXT, 'Processo SEI ORCN - Concluido - Retidos')

In [10]:
navegador.find_element(By.XPATH,'//*[@id="txtTextoPadrao"]').send_keys('Despacho Decisório - Cancelamento de Homologação')
#time.sleep(2)

In [11]:
#CLICA NA PRIMEIRA OPCAO
fc.clica_noelemento(navegador, By.XPATH,'//*[@id="divInfraAjaxtxtTextoPadrao"]/ul/li/a')

In [12]:
#COLOCA O DESPACHO COMO PUBLICO
fc.clica_noelemento(navegador, By.XPATH,'//*[@id="divOptPublico"]/div')

In [13]:
#SALVA DESPACHO
fc.clica_noelemento(navegador, By.ID,'btnSalvar')

In [14]:

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        print('processo achado')
        time.sleep(1)
        #CLICA NO ICONE DE LEGO
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
        
        #navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
        #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
        time.sleep(1.5)
        navegador.switch_to.frame('ifrVisualizacao')
        select_element = navegador.find_element(By.ID, 'selBloco')
        select = Select(select_element)
        #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
        for opcao in select.options:
            if "Despachos para Drones aprovados" in opcao.text:
                select.select_by_visible_text(opcao.text)
                break
        time.sleep(0.5)
        #CLICA NO BOTAO DE INCLUIR NO BLOCO
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
        #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
        



processo achado


In [15]:
#VOLTA PARA PAGINA INICIAL DO PROCESSO
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys('53500.047484/2024-61')
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
navegador.switch_to.frame('ifrConteudoVisualizacao')
time.sleep(1)
#ADICIONA NOTA PARA AGUARDAR ASSINATURA
#CLICA NO ICONE DE ANOTACAO
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
time.sleep(0.1)
#INSERE O TEXTO DA ANOTACAO
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.ID, 'txaDescricao').send_keys('Pedido de Cancelamento')
#SALVA O TEXTO
navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
#DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES

In [16]:
fc.preenche_planilhageral('53500.047484/2024-61', 'André Jacinto Rodrigues', 'Não', 'Cancelado', '', 'Ademir Camargo')

Célula encontrada: nan
